<h3 align="center">Feature Engeneering 6</h3>
<h1 align="center">MONTH_SALES + ITEMS</h1>

### Imports

In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sb
import matplotlib.pyplot as plt
import category_encoders as ce
import pickle

from itertools import product
import calendar 
import gc
import time

from ipywidgets import interact

sb.set()
%matplotlib inline

### Constants

In [2]:
ENGLISH = True

DATA_RUS_PATH = "../DATA/1. Original data Russian (96Mb)/"
DATA_ENG_PATH = "../DATA/2. Translated data English (1Mb)/"
DATA_FE_PATH  = "../DATA/3. Feature Engeneering/"
DATA_SUB_PATH = "../DATA/5. Submissions/"

### Load data

In [3]:
sales = pd.read_csv(DATA_RUS_PATH + "sales_train.csv")          # Dayly sales  Jan 2013 -> Oct 2015
test  = pd.read_csv(DATA_RUS_PATH + "test.csv"                ) # Predict Noviember 2015
sub   = pd.read_csv(DATA_RUS_PATH + "sample_submission.csv", index_col="ID")

if ENGLISH: 
    shops = pd.read_csv(DATA_ENG_PATH + "shops.csv", index_col="shop_id") # shops    (60)
    items = pd.read_csv(DATA_ENG_PATH + "items.csv")           # products  (22170)
    cats  = pd.read_csv(DATA_ENG_PATH + "item_categories.csv") # product categories (84)

else:
    shops = pd.read_csv(DATA_RUS_PATH + "shops.csv", index_col="shop_id")           # shops    (60)
    items = pd.read_csv(DATA_RUS_PATH + "items.csv")           # products  (22170)
    cats  = pd.read_csv(DATA_RUS_PATH + "item_categories.csv") # product categories (84)

In [4]:
def sales_info():
    print("Number of shops:", sales.shop_id.nunique())
    print("Number of items:", sales.item_id.nunique())
    print("Number of sales:", len(sales))
sales_info()

Number of shops: 60
Number of items: 21807
Number of sales: 2935849


<h2 align="center">PART 1: Sales Data Cleaning</h2>

### Remove outliers

In [5]:
print("Sales with negative item_prices:        ", len(sales[ sales['item_price'] < 0]))
print("Sales with item_prices > 50000:         ", len(sales[ sales['item_price'] > 50000]))
print("Sales with negative item_cnt (refounds):", len(sales[ sales['item_cnt_day'] < 0]))
print("Sales with item_cnt > 1000:             ", len(sales[ sales['item_cnt_day'] > 1000]))

Sales with negative item_prices:         1
Sales with item_prices > 50000:          3
Sales with negative item_cnt (refounds): 7356
Sales with item_cnt > 1000:              1


In [6]:
sales = sales.query('0 < item_price < 50000 and 0 < item_cnt_day < 1001')
sales_info()

Number of shops: 60
Number of items: 21802
Number of sales: 2928488


### Remove duplicates

In [7]:
duplicated_shops_replacement = {
    0: 57,
    1: 58,
    11: 10,
    40: 39 # Not sure if this is the same shop
}

def load_obj(name):
    with open(DATA_FE_PATH + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
duplicated_items_replacement = load_obj("duplicated_items_replacement")


sales = sales.replace({
        'shop_id': duplicated_shops_replacement, # replacing obsolete shop id's
        'item_id': duplicated_items_replacement  # fixing duplicate item id's  
})

sales_info()

Number of shops: 56
Number of items: 21356
Number of sales: 2928488


### Removing shops which don't appear in the test set

In [8]:
sales = sales[sales['shop_id'].isin(test.shop_id.unique())]
sales_info()

Number of shops: 42
Number of items: 20693
Number of sales: 2427355


# <center> Date information

### Convert to datatime format

In [9]:
sales['date'] = pd.to_datetime(sales.date,format='%d.%m.%Y')

### Extract time feats

In [10]:
sales['weekday'] = sales.date.dt.dayofweek

#first day the item was sold, day 0 is the first day of the training set period
sales['first_sale_day'] = sales.date.dt.dayofyear 
sales['first_sale_day'] += 365 * (sales.date.dt.year-2013)
sales['first_sale_day'] = sales.groupby('item_id')['first_sale_day'].transform('min').astype('int16')

# ALSO BY groupby('item_id', "shop_id")

In [11]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,weekday,first_sale_day
0,2013-01-02,0,59,22154,999.00,1.0,2,2
1,2013-01-03,0,25,2552,899.00,1.0,3,3
3,2013-01-06,0,25,2554,1709.05,1.0,6,6
4,2013-01-15,0,25,2555,1099.00,1.0,1,15
5,2013-01-10,0,25,2564,349.00,1.0,3,5


### Revenue is needed to accurately calculate prices after grouping


In [12]:
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,weekday,first_sale_day,revenue
0,2013-01-02,0,59,22154,999.00,1.0,2,2,999.00
1,2013-01-03,0,25,2552,899.00,1.0,3,3,899.00
3,2013-01-06,0,25,2554,1709.05,1.0,6,6,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1,15,1099.00
5,2013-01-10,0,25,2564,349.00,1.0,3,5,349.00


### Day Quality -> % of sales per day of the week

We calculate the proportion of weekly sales that occurred on each weekday at each shop. Using this information we can assign a measure of weeks of sales power to each month. February always has 4 exactly weeks worth of days since there are no leap years in our time range and all other months have a value >4 since they have extra days of varying sales power.

Month, year and first day of the month features are also created.

In [36]:
temp = sales.groupby(['shop_id','weekday']).agg({'item_cnt_day':'sum'}).reset_index()
temp.head(7)

,shop_id,weekday,item_cnt_day
0,2,0,3178.0
1,2,1,4209.0
2,2,2,3877.0
3,2,3,3636.0
4,2,4,4559.0
5,2,5,5833.0
6,2,6,5414.0


In [37]:
temp = pd.merge(temp, sales.groupby(['shop_id']).\
                agg({'item_cnt_day':'sum'}).reset_index(), on='shop_id', how='left')

temp.columns = ['shop_id','weekday', 'shop_day_sales', 'shop_total_sales']
temp['day_quality'] = temp['shop_day_sales']/temp['shop_total_sales']
temp = temp[['shop_id','weekday','day_quality']]

temp.head(7)

,shop_id,weekday,day_quality
0,2,0,0.103498
1,2,1,0.137074
2,2,2,0.126262
3,2,3,0.118413
4,2,4,0.148473
5,2,5,0.189963
6,2,6,0.176317


In [42]:
len(temp)

294

### Dates

In [39]:
dates = pd.DataFrame(data={'date':pd.date_range(start='2013-01-01', end='2015-11-30')})

dates['weekday'] = dates.date.dt.dayofweek
dates['month'] = dates.date.dt.month
dates['year'] = dates.date.dt.year - 2013
dates['date_block_num'] = dates['year']*12 + dates['month'] - 1
dates['first_day_of_month'] = dates.date.dt.dayofyear
dates['first_day_of_month'] += 365 * dates['year']

dates

,date,weekday,month,year,date_block_num,first_day_of_month
0,2013-01-01,1,1,0,0,1
1,2013-01-02,2,1,0,0,2
2,2013-01-03,3,1,0,0,3
3,2013-01-04,4,1,0,0,4
4,2013-01-05,5,1,0,0,5
...,...,...,...,...,...,...
1059,2015-11-26,3,11,2,34,1060
1060,2015-11-27,4,11,2,34,1061
1061,2015-11-28,5,11,2,34,1062
1062,2015-11-29,6,11,2,34,1063


### JOIN

In [40]:
dates = dates.join(temp.set_index('weekday'), on='weekday')
dates

,date,weekday,month,year,date_block_num,first_day_of_month,shop_id,day_quality
0,2013-01-01,1,1,0,0,1,2,0.137074
0,2013-01-01,1,1,0,0,1,3,0.115329
0,2013-01-01,1,1,0,0,1,4,0.128662
0,2013-01-01,1,1,0,0,1,5,0.115294
0,2013-01-01,1,1,0,0,1,6,0.136933
...,...,...,...,...,...,...,...,...
1063,2015-11-30,0,11,2,34,1064,55,0.123158
1063,2015-11-30,0,11,2,34,1064,56,0.102228
1063,2015-11-30,0,11,2,34,1064,57,0.114355
1063,2015-11-30,0,11,2,34,1064,58,0.122593


In [44]:
dates = dates.groupby(['date_block_num','shop_id','month','year']).agg({'day_quality':'sum','first_day_of_month':'min'}).reset_index()
dates

,date_block_num,shop_id,month,year,day_quality,first_day_of_month
0,0,2,1,0,4.381749,1
1,0,3,1,0,4.347182,1
2,0,4,1,0,4.373823,1
3,0,5,1,0,4.340164,1
4,0,6,1,0,4.393690,1
...,...,...,...,...,...,...
1465,34,55,11,2,4.267534,1035
1466,34,56,11,2,4.280686,1035
1467,34,57,11,2,4.267389,1035
1468,34,58,11,2,4.273887,1035


In [45]:
dates.query('shop_id == 28').head(15)

,date_block_num,shop_id,month,year,day_quality,first_day_of_month
18,0,28,1,0,4.300037,1
60,1,28,2,0,4.000000,32
102,2,28,3,0,4.605952,60
144,3,28,4,0,4.193211,91
186,4,28,5,0,4.333970,121
228,5,28,6,0,4.472819,152
270,6,28,7,0,4.287644,182
312,7,28,8,0,4.492718,213
354,8,28,9,0,4.313648,244
396,9,28,10,0,4.300037,274


### Convert sales from day datail to month detail

In [46]:
sales = (sales
     .groupby(['date_block_num', 'shop_id', 'item_id'])
     .agg({
         'item_cnt_day':'sum', 
         'revenue':'sum',
         'first_sale_day':'first'
     })
     .reset_index()
     .rename(columns={'item_cnt_day':'item_cnt'})
)
sales.sample(5)

,date_block_num,shop_id,item_id,item_cnt,revenue,first_sale_day
316863,6,57,7682,2.0,598.0,5
135404,2,52,12190,2.0,548.5,2
842676,19,48,21874,1.0,149.0,268
949998,22,53,12363,1.0,999.0,663
690250,15,44,14532,1.0,149.0,31


### 2.3 Constructing Training Dataframe

The test set consists of the **cartesian product** of 42 shops and 5100 items. To make a training set which approximates the test set we create a training dataframe consisting of the cartesian product (active items) x (active shops) for each month.

In [57]:
df = [] 
for block_num in sales['date_block_num'].unique():
    
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    
    df.append(np.array(list(product(cur_shops, cur_items, [block_num]))))

df = pd.DataFrame(np.vstack(df), columns=['shop_id', 'item_id', 'date_block_num'])
df.head()

,shop_id,item_id,date_block_num
0,2,27,0
1,2,33,0
2,2,317,0
3,2,438,0
4,2,471,0


### TEST SET (new data) -> date_block_num = 34

In [61]:
#add the appropriate date_block_num value to the test set
test['date_block_num'] = 34
del test['ID']

### CONCAT TRAIN + TEST for creating feats for both

In [65]:
#append test set to training dataframe
df = pd.concat([df,test]).fillna(0)
df = df.reset_index()
del df['index']
df

,shop_id,item_id,date_block_num
0,2,27,0
1,2,33,0
2,2,317,0
3,2,438,0
4,2,471,0
...,...,...,...
8767791,45,18454,34
8767792,45,16188,34
8767793,45,15757,34
8767794,45,19648,34


# <center> ADD THE COMPUTED USEFUL FEATS

### ADD SALES INFO

In [67]:
#join sales and item inforamtion to the training dataframe
df = pd.merge(df, sales, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)
df

,shop_id,item_id,date_block_num,item_cnt,revenue,first_sale_day
0,2,27,0,1.0,2499.0,2.0
1,2,33,0,1.0,499.0,2.0
2,2,317,0,1.0,299.0,4.0
3,2,438,0,1.0,299.0,19.0
4,2,471,0,2.0,798.0,3.0
...,...,...,...,...,...,...
8767791,45,18454,34,0.0,0.0,0.0
8767792,45,16188,34,0.0,0.0,0.0
8767793,45,15757,34,0.0,0.0,0.0
8767794,45,19648,34,0.0,0.0,0.0


### ADD DATE INFO

In [68]:
dates

,date_block_num,shop_id,month,year,day_quality,first_day_of_month
0,0,2,1,0,4.381749,1
1,0,3,1,0,4.347182,1
2,0,4,1,0,4.373823,1
3,0,5,1,0,4.340164,1
4,0,6,1,0,4.393690,1
...,...,...,...,...,...,...
1465,34,55,11,2,4.267534,1035
1466,34,56,11,2,4.280686,1035
1467,34,57,11,2,4.267389,1035
1468,34,58,11,2,4.273887,1035


In [69]:
df = pd.merge(df, dates, on=['date_block_num','shop_id'], how='left')
df

,shop_id,item_id,date_block_num,item_cnt,revenue,first_sale_day,month,year,day_quality,first_day_of_month
0,2,27,0,1.0,2499.0,2.0,1,0,4.381749,1
1,2,33,0,1.0,499.0,2.0,1,0,4.381749,1
2,2,317,0,1.0,299.0,4.0,1,0,4.381749,1
3,2,438,0,1.0,299.0,19.0,1,0,4.381749,1
4,2,471,0,2.0,798.0,3.0,1,0,4.381749,1
...,...,...,...,...,...,...,...,...,...,...
8767791,45,18454,34,0.0,0.0,0.0,11,2,4.288065,1035
8767792,45,16188,34,0.0,0.0,0.0,11,2,4.288065,1035
8767793,45,15757,34,0.0,0.0,0.0,11,2,4.288065,1035
8767794,45,19648,34,0.0,0.0,0.0,11,2,4.288065,1035


### ADD SHOP INFO

In [ ]:
df = pd.merge(df, items.drop(columns=['item_name','group_name','category_name']), on='item_id', how='left')